<a href="https://colab.research.google.com/github/sriharikrishna/EuroAD26/blob/main/EuroAD_tf_odetest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`solve` is provided by the `tensorflow_probability` package.

In [ ]:
import tensorflow_probability as tfp
import tensorflow as tf

The solver call works as follows. (This example does not have a wrapper)

In [ ]:
y0 = tf.Variable([2.0 , 6.], shape=(2,), dtype=tf.float64)
t_i, t_m, t_f = 0., 0.5, 1.
A = tf.Variable([[2., 4.], [3., 2.]], dtype = tf.float64)

def ode_fn(_: float, y: tf.Tensor, A: tf.Tensor) -> tf.Tensor:
    val = tf.linalg.matvec(A, y)
    return val

resultsfull = tfp.math.ode.DormandPrince().solve(ode_fn, t_i, y0,
                              solution_times = tfp.math.ode.ChosenBySolver(final_time=t_f),
                              #solution_times = [t_m, t_f],
                              constants={"A": A})
print(resultsfull.states[-1])

The code is modified to compute the Jacobian. Importantly, we cannot express `t_i`  and `t_f` as variables, so we cannot get derivatives w.r.t. them.

In [ ]:
y0 = tf.Variable([2.0 , 6.], shape=(2,), dtype=tf.float64)
t_i, t_m, t_f = 0., 0.5, 1.
A = tf.Variable([[2., 4.], [3., 2.]], dtype = tf.float64)

def ode_fn(_: float, y: tf.Tensor, A: tf.Tensor) -> tf.Tensor:
    val = tf.linalg.matvec(A, y)
    return val

with tf.GradientTape(persistent=True) as tape:
    tape.watch(y0)
    resultsfull = tfp.math.ode.DormandPrince().solve(ode_fn, t_i, y0,
                              solution_times = tfp.math.ode.ChosenBySolver(final_time=t_f),
                              #solution_times = [t_m, t_f],
                              constants={"A": A})
print(resultsfull.states[-1])
jac = tape.jacobian(resultsfull.states, y0, experimental_use_pfor=False)
print(jac)

Running the code with complex data types leads to an error.

In [ ]:
import tensorflow_probability as tfp
import tensorflow as tf

y0 = tf.Variable([2.0 , 6.], shape=(2,), dtype=tf.complex128)
t_i, t_m, t_f = 0., 0.5, 1.
A = tf.Variable([[2., 4.], [3., 2.]], dtype = tf.complex128)

def ode_fn(_: float, y: tf.Tensor, A: tf.Tensor) -> tf.Tensor:
    val = tf.linalg.matvec(A, y)
    return val

with tf.GradientTape(persistent=True) as tape:
    tape.watch(y0)
    resultsfull = tfp.math.ode.DormandPrince().solve(ode_fn, t_i, y0,
                              solution_times = tfp.math.ode.ChosenBySolver(final_time=t_f),
                              #solution_times = [t_m, t_f],
                              constants={"A": A})
print(resultsfull.states[-1])
jac = tape.jacobian(resultsfull.states, y0, experimental_use_pfor=False)
print(jac)

So we need to look elsewhere.